<a href="https://colab.research.google.com/github/hwangsaeyeon/AAI-Web-Development/blob/main/news_data_crawling_extension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#이 부분은 처음 한번만 실행하면 됨
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 4.9 MB/s 
     |████████████████████████████████| 384 kB 42.1 MB/s 
     |████████████████████████████████| 140 kB 29.4 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machin

In [ ]:
from selenium.webdriver.common.by import By
import pandas as pd 

In [ ]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
 
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)


#해당 url로 이동
url = "https://m.entertain.naver.com/ranking?rankingDate=202209" #네이버뉴스 연예 랭킹 페이지
#date = [str(0)+str(d) for d in range(1,10)] + [str(d) for d in range(10,31)]
data = list()
date = ['01']
for i in range(len(date)): 
    driver.get(url+date[i]) #2022년 09월 nn일 랭킹 페이지
    print(date[i]+'일 네이버 연예 기사 랭킹 페이지 url: ',driver.current_url)
    print()
    for j in range(1,31): #1~30위 랭킹 기사 크롤링
        
        #조회수
        views = '//*[@id="ct"]/div/div[2]/ul/li[1]/a/div[1]/span'
        selector = driver.find_element(By.XPATH, views)
        print(selector.text[3:])

        #기사 본문으로 접속
        article_page = '#ct > div > div.com_ranking.index > ul > li:nth-child('+str(j)+') > a'
        selector = driver.find_element(By.CSS_SELECTOR, article_page)
        href = selector.get_attribute('href')
        driver.get(href)

        #기사 타이틀 클롤링
        title = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_title > h2')

        #기사 날짜 
        article_date = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span')
        print(article_date.text)

        #기자 
        reporter = driver.find_element(By.CSS_SELECTOR, '#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_journalist > a > em')
        print(reporter.text)


        #기자 구독자
        subscribers = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div[1]/div[1]/div[2]/div[3]/div/div/div/div[1]/div/div[1]/div[2]/div/div/a/span[3]/span[1]/em')
        #print(subscribers.text)


        #기자 응원수 
        #reporter_cheers = driver.find_element(By.XPATH, '/html/body/div/div[2]/div/div[1]/div[1]/div[2]/div[3]/div/div/div/div[1]/div/div[1]/div[2]/div/div/a/span[3]/span[2]/div/span/em[2]')
        #print(reporter_cheers.text)

        #좋아요 수, 응원해요 수, 축하해요 수, 기대해요 수, 놀랐어요 수, 슬퍼요 수 
        good = driver.find_element(By.CSS_SELECTOR, '#likeItCountViewDiv > ul > li.u_likeit_list.good > a > span.u_likeit_list_count._count')
        cheer = driver.find_element(By.CSS_SELECTOR, '#likeItCountViewDiv > ul > li.u_likeit_list.cheer > a > span.u_likeit_list_count._count')
        congrats = driver.find_element(By.CSS_SELECTOR, '#likeItCountViewDiv > ul > li.u_likeit_list.congrats > a > span.u_likeit_list_count._count')
        expect = driver.find_element(By.CSS_SELECTOR, '#likeItCountViewDiv > ul > li.u_likeit_list.expect > a > span.u_likeit_list_count._count')
        surprise = driver.find_element(By.CSS_SELECTOR, '#likeItCountViewDiv > ul > li.u_likeit_list.surprise > a > span.u_likeit_list_count._count')
        sad  = driver.find_element(By.CSS_SELECTOR, '#likeItCountViewDiv > ul > li.u_likeit_list.sad > a > span.u_likeit_list_count._count')
        print(good.text)
        print(cheer.text)
        print(congrats.text)
        print(expect.text)
        print(surprise.text)
        print(sad.text)


        #기사 추천수
        article_recommend = driver.find_element(By.CSS_SELECTOR, '#toMainContainer > a > em.u_cnt._count')
        print(article_recommend.text)

        #기사 본문 크롤링 
        body = driver.find_element(By.CSS_SELECTOR, '#newsct_article')
        
        #기사 요약문 
        button = driver.find_element(By.CSS_SELECTOR, '#_SUMMARY_BUTTON > a')
        button.click()
        summary = driver.find_element(By.CSS_SELECTOR, '#_SUMMARY_BUTTON > div.media_end_head_autosummary_layer._auto_summary_contents._SUMMARY_LAYER > div.media_end_head_autosummary_layer_body > div')
        print(summary.text)
        #close = driver.find_element(By.CSS_SELECTOR, '#_SUMMARY_BUTTON > div.media_end_head_autosummary_layer._auto_summary_contents._SUMMARY_LAYER > div.media_end_head_autosummary_layer_head > a')
        #close.click()

        data.append([title.text, body.text])
        driver.back() #랭킹 메인화면 페이지로 돌아감

news_crawling = pd.DataFrame(data, columns=['Title', 'Body'] )
news_crawling.to_csv('news_crawling_2022_09.csv',)

01일 네이버 연예 기사 랭킹 페이지 url:  https://m.entertain.naver.com/ranking?rankingDate=20220901


368,807
2022.09.01. 오전 8:03
우빈 기자
0
0
0
0
0
0
0
0



368,807
2022.09.01. 오전 1:20
선미경 기자
0
0
0
0
0
0
0
0



368,807
2022.09.01. 오후 4:42
정서희 기자
0
0
0
0
0
7
589
390
103


368,807
2022.09.01. 오전 7:03
배재련 기자
0
0
0
0
27
7
422
53
108


368,807
2022.09.01. 오후 2:33
조지영 기자
0
0
0
0
0
0
0
0



368,807
2022.09.01. 오후 3:24
김지혜 기자
0
0
0
0
0
0
0
0



368,807
2022.09.01. 오전 9:53
태유나 기자
0
0
0
0
0
0
0
0



368,807
2022.09.01. 오후 3:07
정안지 기자
0
0
0
0
0
0
79
19
12


368,807
2022.09.01. 오전 7:52
황수연 기자
0
0
0
0
0
0
112
17
10


368,807
2022.09.01. 오후 6:21
이예진 기자


KeyboardInterrupt: ignored